In [1]:
import os
import glob
import io
import imageio
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib import colors
from pysteps import nowcasts
from pysteps.motion.lucaskanade import dense_lucaskanade
from pysteps.utils import transformation

# --- KONFIGURASI ---
INPUT_TIF_FOLDER = 'input/tif'
OUTPUT_GIF_FILE = 'nowcast_animation.gif'
NUM_INPUT_FILES = 12
N_LEADTIMES = 18
TIMESTEP = 10
KM_PER_PIXEL = 2.2269
N_ENS_MEMBERS = 20

def create_animation(nowcast_frames_mmh, metadata, output_filename, timestep, last_observation_mmh):
    """
    Membuat dan menyimpan animasi GIF dari rangkaian frame prediksi nowcast.
    """
    print(f"\nMembuat animasi GIF: {output_filename}...")
    
    images = []
    ensemble_mean_frames = np.mean(nowcast_frames_mmh, axis=0)
    all_frames = np.concatenate(([last_observation_mmh], ensemble_mean_frames), axis=0)
    extent = (
        metadata['geodata']['x1'], metadata['geodata']['x2'],
        metadata['geodata']['y1'], metadata['geodata']['y2']
    )
    precip_colors = [
        "#f0f8ff", "#87cefa", "#1e90ff", "#00ff00",
        "#ffff00", "#ffc800", "#ff8000", "#ff0000", "#8b0000"
    ]
    cmap = colors.ListedColormap(precip_colors)
    levels = [0.1, 0.5, 1, 2, 5, 10, 20, 50]
    norm = colors.BoundaryNorm(levels, cmap.N)
    for i, frame in enumerate(all_frames):
        if i == 0:
            title = "Observasi Terakhir (T+0)"
        else:
            lead_time = i * timestep
            title = f"Prediksi T+{lead_time} menit"
        fig, ax = plt.subplots(figsize=(10, 8))
        masked_frame = np.ma.masked_less(frame, 0.1)
        img = ax.imshow(masked_frame, extent=extent, cmap=cmap, norm=norm, origin='upper', interpolation='none')
        ax.set_title(title, fontsize=16)
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        cbar = fig.colorbar(img, ax=ax, ticks=levels, extend='max', spacing='proportional')
        cbar.set_label('Intensitas Curah Hujan (mm/jam)', fontsize=12)
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        images.append(imageio.imread(buf))
        plt.close(fig)
    imageio.mimsave(output_filename, images, duration=5, loop=0)
    print(f"Animasi berhasil disimpan ke '{output_filename}'")


def main():
    """
    Fungsi utama untuk menjalankan alur kerja nowcasting dan membuat animasi.
    """
    try:
        all_tif_files = glob.glob(os.path.join(INPUT_TIF_FOLDER, '*.tif'))
        def get_timestamp_key(filepath):
            try:
                return os.path.basename(filepath).split('_')[1].split('.')[0]
            except IndexError: return "0"
        
        tif_files = sorted(all_tif_files, key=get_timestamp_key)
        input_files = tif_files[-NUM_INPUT_FILES:]
        if len(input_files) < 3:
             print("Error: Kurang dari 3 file ditemukan. Tidak dapat melanjutkan.")
             return
    except Exception as e:
        print(f"Error saat mencari file: {e}")
        return

    R = []
    metadata = {}
    for file_path in input_files:
        with rasterio.open(file_path) as ds:
            R.append(ds.read(1))
            if file_path == input_files[-1]:
                metadata['geodata'] = {'projection': ds.crs.to_proj4(), 'x1': ds.bounds.left, 'y1': ds.bounds.bottom, 'x2': ds.bounds.right, 'y2': ds.bounds.top, 'yorigin': 'upper'}
    R = np.stack(R)

    # --- 2. Pra-pemrosesan Data ---
    R, metadata_db = transformation.dB_transform(R, threshold=0.1, zerovalue=-15.0)
    R[~np.isfinite(R)] = -15.0
    
    # --- 3. Hitung dan Stabilkan Vektor Gerak ---
    V = dense_lucaskanade(R[-3:, :, :]) 
    V[~np.isfinite(V)] = 0.0
    max_velocity = 100
    V[V > max_velocity] = max_velocity
    V[V < -max_velocity] = -max_velocity
    
    # --- 4. Hasilkan Nowcast menggunakan STEPS ---
    nowcast_method = nowcasts.get_method("steps")
    R_f = nowcast_method(
        R, V, N_LEADTIMES, N_ENS_MEMBERS,
        kmperpixel=KM_PER_PIXEL, timestep=TIMESTEP, precip_thr=-10.0,
        seed=42, extrap_kwargs={'boundary_condition': 'zero'},
        noise_method='parametric', ar_order=1
    )
    print("Nowcast selesai.")

    # --- 5. Visualisasi ---
    R_f_mmh = transformation.dB_transform(R_f, threshold=-10.0, inverse=True)[0]
    last_observation_mmh = transformation.dB_transform(R[-1,:,:], threshold=-10.0, inverse=True)[0]
    create_animation(R_f_mmh, metadata, OUTPUT_GIF_FILE, TIMESTEP, last_observation_mmh)


if __name__ == "__main__":
    main()



Pysteps configuration file found at: /home/nugrahab/.conda/envs/pysteps/lib/python3.11/site-packages/pysteps/pystepsrc

Inputs validated and initialized successfully.
Computing STEPS nowcast
-----------------------

Inputs
------
input dimensions: 500x550
km/pixel:         2.2269
time step:        10 minutes

Methods
-------
extrapolation:          semilagrangian
bandpass filter:        gaussian
decomposition:          fft
noise generator:        parametric
noise adjustment:       no
velocity perturbator:   bps
conditional statistics: no
precip. mask method:    incremental
probability matching:   cdf
FFT method:             numpy
domain:                 spatial

Parameters
----------
number of time steps:     18
ensemble size:            20
parallel threads:         1
number of cascade levels: 6
order of the AR(p) model: 1
velocity perturbations, parallel:      10.88,0.23,-7.68
velocity perturbations, perpendicular: 5.76,0.31,-2.72
precip. intensity threshold: -10.0
Nowcast components 

/tmp/ipykernel_4800/3444852050.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(buf))


Animasi berhasil disimpan ke 'nowcast_animation.gif'
